In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/function/myfunc_baseline")
sys.path.append("/content/drive/MyDrive/Colab Notebooks/function/myfunc_baseline/dataset")

In [ ]:
import torch
import time
import os
import torch.optim as optim
import numpy as np
import dataset
import csv
from model import LinearNet,Network
from dataset import get_loader
from scipy.io import savemat
from criteria import hLoss, rLoss, oError, Conv, avgPre

device = ('cuda' if torch.cuda.is_available() else 'cpu')

import matplotlib.pyplot as plt
import pandas as pd
def TablePlot(df,w,h,outputPath):
    fig, ax = plt.subplots(figsize=(w,h))
    ax.axis('off')
    ax.table(
        df.values,
        #colLabels = df.columns,
        loc = 'center',
        bbox=[0,0,1,1],
        fontsize = 100
    )
    plt.savefig(outputPath)
    plt.show()
def criterion(predictions, confidence):

    assert predictions.shape == confidence.shape

    N, C = confidence.shape

    loss_sum = torch.zeros(N, 1, dtype=torch.float, device=device)

    ones = torch.ones_like(confidence, dtype=torch.float, device=device)
    zeros = torch.zeros_like(confidence, dtype=torch.float, device=device)

    for i in range(C):
        confidence_i = confidence[:, i].view(N, -1)
        prediction_i = predictions[:, i].view(N, -1)

        loss = torch.max(zeros, confidence_i - confidence) * torch.max(zeros, ones - (prediction_i - predictions))
        loss_sum += torch.sum(loss, dim=-1, keepdim=True) / ((C - 1) * 1.0)

    return loss_sum.mean()

def test(net, test_loader):

    ######################################
    #信頼度のしきい値
    threshold = 0.8
    ######################################

    net.eval()

    hLoss_list = []
    rLoss_list = []
    oError_list = []
    conv_list = []
    avgPre_list = []

    for itr, (inputs, labels) in enumerate(test_loader):

        inputs = inputs.to(device)
        outputs = net(inputs)
        prelabel = (torch.sigmoid(outputs) > threshold).float()

        hLoss_list.append(hLoss(outputs, labels))
        rLoss_list.append(rLoss(outputs, labels))
        oError_list.append(oError(outputs, labels))
        conv_list.append(Conv(outputs, labels))
        avgPre_list.append(avgPre(outputs, labels))

    hamming_loss = np.mean(hLoss_list)
    ranking_loss = np.mean(rLoss_list)
    one_error = np.mean(oError_list)
    coverage = np.mean(conv_list)
    avg_precision = np.mean(avgPre_list)

    return hamming_loss, ranking_loss, one_error, coverage, avg_precision, outputs.cpu().detach().numpy() ,prelabel.cpu().numpy()

def train_common(net, optimizer, train_loader):
    train_loss = 0

    for batch_idx, (inputs, labels, creds) in enumerate(train_loader):
        net.train()
        inputs, labels, creds = inputs.to(device), labels.to(device), creds.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, creds)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    return train_loss

def train_meta(net, optimizer, train_loader, meta_loader):
    train_loss = 0.0
    meta_loss = 0.0
    clean_iter = iter(meta_loader)
    for batch_idx, (inputs, labels,creds) in enumerate(train_loader):
        inputs, labels,creds = inputs.to(device), labels.to(device),creds.to(device)
        meta_net = LinearNet(num_inputs=features_num, num_outputs=labels_num)
        meta_net.load_state_dict(net.state_dict())
        meta_net.to(device)
        epsilon = torch.zeros_like(labels, requires_grad=True)

        y_f_hat = meta_net(inputs)
        l_f_meta = criterion(y_f_hat, epsilon)
        meta_net.zero_grad()

        grads = torch.autograd.grad(l_f_meta, (meta_net.params()), create_graph=True)
        meta_net.update_params(lr, source_params=grads)

        try:
            val_data, val_labels,val_creds = next(clean_iter)
        except StopIteration:
            clean_iter = iter(meta_loader)
            val_data, val_labels,val_creds = next(clean_iter)

        val_data, val_labels = val_data.to(device), val_labels.to(device)

        y_g_hat = meta_net(val_data)
        l_g_meta = criterion(y_g_hat, val_labels)

        grad_eps = torch.autograd.grad(l_g_meta, epsilon, only_inputs=True)[0]

        # computing and normalizing the confidence matrix P
        # p_tilde = torch.clamp(epsilon - grad_eps, min=0)
        p_tilde = torch.clamp(-grad_eps, min=0)
        p_tilde *= labels

        max_row = torch.max(p_tilde, dim=1, keepdim=True)[0]
        ones = torch.ones_like(max_row)
        max_row = torch.where(max_row == 0, ones, max_row)
        p = p_tilde / max_row
        # p.shape = batch_size * class_num

        y_f_hat = net(inputs)
        l_f = criterion(y_f_hat, p)

        optimizer.zero_grad()
        l_f.backward()
        optimizer.step()

        meta_loss += l_g_meta.item()
        train_loss += l_f.item()

    return meta_loss, train_loss


def baseline(train_loader, test_loader, meta_loader, num_epoch=300, clean=True):
    # method = baseline for baseline with meta data, else for baseline without meta data and ground-truth

    net = LinearNet(num_inputs=features_num, num_outputs=labels_num)
    #net = Network(num_inputs=features_num,num_hides = 5,num_outputs=labels_num)
    net = net.to(device)
    optimizer = optim.SGD(params=net.params(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    hLoss_list = []
    rLoss_list = []
    oError_list = []
    conv_list = []
    avgPre_list = []

    for epoch in range(num_epoch):
        loss = 0.0
        # adjust_learning_rate(optimizer, epoch, lr)
        if clean:
            loss += train_common(net, optimizer, meta_loader)
            loss += train_common(net, optimizer, train_loader)
            loss /= (len(meta_loader) + len(train_loader))
        else:
            loss += train_common(net, optimizer, train_loader)
            loss /= len(train_loader)

        hamming_loss, ranking_loss, one_error, coverage, avg_precision, outputs, p_labels = test(net, test_loader)

        print('Batch: [{:0>4}/{:0>4}] '.format(epoch + 1, num_epoch),
              'training loss: {:.4f} '.format(loss),
              'hLoss: {:.4f} '.format(hamming_loss),
              'rLoss: {:.4f} '.format(ranking_loss),
              'oError: {:.4f} '.format(one_error),
              'conv: {:.4f} '.format(coverage),
              'avgPre: {:.4f}'.format(avg_precision))

    #最終的な結果と出力と予測ラベルを返す
    return hamming_loss, ranking_loss, one_error, coverage, avg_precision, outputs, p_labels

def metalearning(train_loader, test_loader, meta_loader,train_data,train_plabels, meta_features, meta_labels,data,p_true,num_epoch=300, clean=True,save_creds_every_epoch=False):
    
    net = LinearNet(num_inputs=features_num, num_outputs=labels_num)
    net = net.to(device)
    optimizer = optim.SGD(params=net.params(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    hLoss_list = []
    rLoss_list = []
    oError_list = []
    conv_list = []
    avgPre_list = []

    for epoch in range(num_epoch):
        train_loss = 0.0
        meta_loss = 0.0

        if clean:
            train_loss += train_common(net, optimizer, train_loader)
            train_l, meta_l = train_meta(net, optimizer, train_loader, meta_loader)
            train_loss += train_l
            meta_loss += meta_l

            train_loss /= (len(train_loader) + len(meta_loader))
            meta_loss /= len(train_loader)
        else:
            train_l, meta_l = train_meta(net, optimizer, train_loader, meta_loader)
            train_loss += train_l
            meta_loss += meta_l

            train_loss /= len(train_loader)
            meta_loss /= len(train_loader)

        if save_creds_every_epoch == True:
          p_creds = compute_p_creds(train_data,train_plabels, meta_features, meta_labels,net)
          np.savetxt(f"/content/drive/MyDrive/Colab Notebooks/result/predict_creds/PMLMD_every_epoch/{data}/{str(p_true)}/{epoch}.csv",p_creds,delimiter = ",")

        hamming_loss, ranking_loss, one_error, coverage, avg_precision ,outputs, p_labels= test(net, test_loader)

        print('Batch: [{:0>4}/{:0>4}] '.format(epoch + 1, num_epoch),
              'training loss: {:.4f} '.format(train_loss),
              'meta_loss: {:.4f} '.format(meta_loss),
              'hLoss: {:.4f} '.format(hamming_loss),
              'rLoss: {:.4f} '.format(ranking_loss),
              'oError: {:.4f} '.format(one_error),
              'conv: {:.4f} '.format(coverage),
              'avgPre: {:.4f}'.format(avg_precision))

    #最終的な結果と出力と予測ラベルを返す
    return hamming_loss, ranking_loss, one_error, coverage, avg_precision, outputs, p_labels, net

#メタ学習時の信頼度の推定
def compute_p_creds(train_data,train_plabels, meta_features, meta_labels, net):
    # 最終的なネットワークをもとに訓練データ全体の信頼度を計算
    inputs = train_data
    labels = train_plabels
    inputs, labels = inputs.to(device), labels.to(device)

    meta_net = LinearNet(num_inputs=features_num, num_outputs=labels_num)
    meta_net.load_state_dict(net.state_dict())
    meta_net.to(device)
    epsilon = torch.zeros_like(labels, requires_grad=True)

    y_f_hat = meta_net(inputs)
    l_f_meta = criterion(y_f_hat, epsilon)
    meta_net.zero_grad()

    grads = torch.autograd.grad(l_f_meta, (meta_net.params()), create_graph=True)
    meta_net.update_params(lr, source_params=grads)

    val_data = meta_features
    val_labels = meta_labels
    val_data, val_labels = val_data.to(device), val_labels.to(device)

    y_g_hat = meta_net(val_data)
    l_g_meta = criterion(y_g_hat, val_labels)

    grad_eps = torch.autograd.grad(l_g_meta, epsilon, only_inputs=True)[0]

    # computing and normalizing the confidence matrix P
    # p_tilde = torch.clamp(epsilon - grad_eps, min=0)
    p_tilde = torch.clamp(-grad_eps, min=0)
    p_tilde *= labels

    max_row = torch.max(p_tilde, dim=1, keepdim=True)[0]
    ones = torch.ones_like(max_row)
    max_row = torch.where(max_row == 0, ones, max_row)
    p = p_tilde / max_row
    p = p.cpu().numpy()
    return p
    
def save_p_creds(data,train_target,p_noise,p_true,cv_num,creds):
  save_path = f"/content/drive/MyDrive/Colab Notebooks/result/predict_creds/PML-MD/{data}/{str(p_noise)}/true/{str(p_true)}/"
  np.savetxt(save_path+"creds_"+str(cv_num)+".csv",creds,delimiter=",")
  np.savetxt(save_path+"target_"+str(cv_num)+".csv",train_target,delimiter=",")

def save_result(data, method, learning_rate, result):
    file = './result/LinearNet/' + data
    if not os.path.exists(file):
        os.mkdir(file)
    filename = './result/LinearNet/' + data + '/' + method + '_' + str(learning_rate) + '.mat'
    content = {}

    content['result'] = result
    savemat(filename, content)

def save_log(content, learning_rate):
    filename = './result/LinearNet/' + str(learning_rate) + '_log.txt'
    file = open(filename, 'a')
    file.write(content)
    file.close()

def adjust_learning_rate(optimizer, epochs, learning_rate):
    lr = learning_rate * ((0.1 ** int(epochs >= 80)) * (0.1 ** int(epochs >= 100)))  # For WRN-28-10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# 訓練データを取る
def get_train_data(dataname,p_noise,p_true,cv_num):
  data_path = f"/content/drive/MyDrive/Colab Notebooks/new_data2/" + dataname + "/"
  features = np.loadtxt(data_path+"data.csv", delimiter=',')
  labels = np.loadtxt(data_path+"target.csv", delimiter=',',dtype = float)
  if dataname in ['mirflickr', 'music_emotion', 'music_style','YeastBP']:
    plabels = np.loadtxt(data_path+"cand/0.csv", delimiter=',',dtype = float)
  else:
    plabels = np.loadtxt(data_path+"cand/"+str(p_noise)+".csv", delimiter=',',dtype = float)
  cv_inds = np.loadtxt(data_path+"index/5-cv.csv",delimiter=',',dtype = int)-1
  data_path_true = "/content/drive/MyDrive/Colab Notebooks/new_data/" + dataname + "/index/true/" + str(p_true) +  "/A.csv"
  with open(data_path_true) as f:
      reader = csv.reader(f)
      l = [row for row in reader]
  meta_inds = [[int(v) for v in row] for row in l]

  #全体の特徴をtensorに変換
  features = torch.tensor(features, dtype=torch.float)
  labels = torch.tensor(labels, dtype=torch.float)
  plabels = torch.tensor(plabels, dtype=torch.float)

  zeros = torch.zeros_like(labels)
  labels = torch.where(labels == -1, zeros, labels)
  plabels = torch.where(plabels == -1, zeros, plabels)

  #訓練とテストに分割(自分は5分割交差検証)
  # split training into train and test set
  """n = len(features)
  train_size = int(n * prec)"""
  
  #trainとtestとmetaのインデックス(自分はcv_inds)
  """indices = torch.randperm(n)
  train_idxs = indices[:train_size]
  test_idxs = indices[train_size:]"""
  index = np.where(cv_inds!=cv_num)
  train_inds = np.ravel(index[0])
  train_idxs = torch.tensor(train_inds)
  
  index = np.where(cv_inds==cv_num)
  index = np.ravel(index[0])
  test_idxs = torch.tensor(index)
  

  #上で指定したインデックスでtrainとtestに分ける             
  train_features = torch.index_select(features, 0, train_idxs)
  train_labels = torch.index_select(labels, 0, train_idxs)
  train_plabels = torch.index_select(plabels, 0, train_idxs)

  clean_features = train_features.clone()
  clean_labels = train_labels.clone()

  noisy_features = train_features.clone()
  noisy_labels = train_plabels.clone()

  test_features = torch.index_select(features, 0, test_idxs)
  test_labels = torch.index_select(labels, 0, test_idxs)

  
  #検証集合を作る
  # select a clean batch from training set
  meta_idxs = np.array([],dtype = "int")
  for i in range(5):
    if i != cv_num:
      meta_idxs = np.append(meta_idxs,np.array(meta_inds[i],dtype="int"))

  if dataname in ["music_emotion","music_style"]:
    batch_size = 200
  elif dataname in ["mirflickr"]:
    batch_size = 500
  elif dataname in ['CAL500','emotions','genbase']:
    batch_size = 50
  elif dataname in ['scene','enron']:
    batch_size = 100

  train_idxs = np.setdiff1d(train_inds,meta_idxs)
  train_idxs = torch.tensor(train_idxs)
  meta_idxs = torch.tensor(meta_idxs)

  meta_features = torch.index_select(features, 0, meta_idxs)
  meta_labels = torch.index_select(labels, 0, meta_idxs)
  
  train_features = torch.index_select(features, 0, train_idxs)
  train_target = torch.index_select(labels, 0, train_idxs)
  train_plabels = torch.index_select(plabels, 0, train_idxs)
  
  return train_features, train_target,  train_plabels, meta_features, meta_labels

In [ ]:
import importlib
importlib.reload(dataset)

<module 'dataset' from '/content/drive/MyDrive/Colab Notebooks/function/myfunc_baseline/dataset.py'>

In [ ]:
dir = "/content/drive/MyDrive/Colab Notebooks"
batch_size = 100
meta_size = 5
#lr_list = [0.01, 0.001]
lr_list = [0.03]
momentum = 0.9
weight_decay = 1e-4
#methods = ['meta','baseline', 'baseline_clean','ground_truth']
#datasets = ['music_style','mirflickr','emotions','enron','emotions','CAL500','scene','genbase']
#datasets  =['emotions','enron','CAL500','scene','genbase']
datasets = ['music_emotion']
method='meta' # chose from meta,baseline_clean,baseline
cv_start = 0
cv_end = 1
noise_list_m = [50,100,150]
#noise_list_m = [150]
true_list = [1,3,5,7,9]
true_list = [1,9]
do_save_creds = False
if __name__ == '__main__':

    #for data in datasets:
    for data in datasets:
        print('\n')

        if data in ['music_emotion', 'music_style']:
            batch_size = 200
        elif data in ['mirflickr']:
            batch_size = 500
        elif data in ['CAL500', 'emotions','genbase']:
            batch_size = 50
        elif data in ['enron','scene']:
            batch_size = 100

        if data in ['music_emotion', 'music_style','mirflickr','YeastBP']:
          noise_list = [0]
        elif data in ['emotions','enron','CAL500','scene','genbase']:
          noise_list = noise_list_m.copy()

        for lr in lr_list:
            #for method in methods:
            #print('Dataname: {}\t Methods: {}\t Learning rate: {}'.format(data, method, lr))
            #result = np.empty((repeat, 5), dtype=np.float)
            for i in range(cv_start,cv_end):
                for p_noise in noise_list:
                    for tru in range(len(true_list)):
                    #for p_true in true_list:
                        p_true = true_list[tru]
                        cv_num = i
                        print(cv_num)
                        print(f'data={data}/cv={str(i)}/p_noise={str(p_noise)}/p_true={str(p_true)}/method={method}\n')
                        train_loader, creds_loader, test_loader, meta_loader, clean_loader, \
                        noisy_loader, features_num, labels_num = dataset.get_loader(data, batch_size, cv_num,p_noise,p_true,meta_size=meta_size)

                        if method == 'meta':
                            train_data, train_target,  train_plabels, meta_features, meta_labels = get_train_data(data,p_noise,p_true,cv_num)
                            hamming_loss, ranking_loss, one_error, coverage, avg_precision ,outputs, p_labels ,net= metalearning(train_loader,
                                                                                                    test_loader,
                                                                                                    meta_loader,
                                                                                                    train_data,
                                                                                                    train_plabels,
                                                                                                    meta_features,
                                                                                                    meta_labels,
                                                                                                    data,
                                                                                                    p_true,
                                                                                                    clean=False,
                                                                                                    save_creds_every_epoch=True)
                            if do_save_creds:
                              p_creds = compute_p_creds(train_data,train_plabels, meta_features, meta_labels,net)
                              save_p_creds(data,train_target,p_noise,p_true,cv_num,p_creds)


                        elif method == 'meta_clean':
                            hamming_loss, ranking_loss, one_error, coverage, avg_precision ,outputs, p_labels = metalearning(train_loader,
                                                                                                    test_loader,
                                                                                                    meta_loader,
                                                                                                    clean=True)
                        elif method == 'baseline':
                            hamming_loss, ranking_loss, one_error, coverage, avg_precision ,outputs, p_labels = baseline(noisy_loader,
                                                                                                test_loader, 
                                                                                                meta_loader,
                                                                                                clean=False)
                            
                        # myfunc train_loader to creds_loader
                        # 提案手法
                        elif method == 'baseline_clean':
                            hamming_loss, ranking_loss, one_error, coverage, avg_precision ,outputs, p_labels = baseline(creds_loader,
                                                                                                test_loader, 
                                                                                                meta_loader,
                                                                                                clean=True)

                        elif method == 'ground_truth':           # ground truth
                            hamming_loss, ranking_loss, one_error, coverage, avg_precision ,outputs, p_labels = baseline(clean_loader,
                                                                                                test_loader, 
                                                                                                meta_loader,
                                                                                                clean=False)

                        #np.savetxt(f"{dir}/result/myfunc_with_regu/{data}/{str(p_noise)}/true/{str(p_true)}/creds_{str(i)}.csv",outputs,delimiter =',')
                        #np.savetxt(f"{dir}/result/myfunc_with_regu/{data}/{str(p_noise)}/true/{str(p_true)}/predict_labels_{str(i)}.csv",p_labels,fmt='%d',delimiter=',')


                        print()
                        print('Test results of the last :\t',
                            'Best hLoss: {:.4f} '.format(hamming_loss),
                            'Best rLoss: {:.4f} '.format(ranking_loss),
                            'Best oError: {:.4f} '.format(one_error),
                            'Best conv: {:.4f} '.format(coverage),
                            'Best avgPre: {:.4f} '.format(avg_precision))



0
data=music_emotion/cv=0/p_noise=0/p_true=1/method=meta

W:(5412, 54)
train:(5412, 98)
meta:(54, 98)
Batch: [0001/0300]  training loss: 1.5716  meta_loss: 2.1236  hLoss: 0.2198  rLoss: 0.3401  oError: 0.6650  conv: 0.5115  avgPre: 0.4960
Batch: [0002/0300]  training loss: 1.2881  meta_loss: 1.5424  hLoss: 0.2174  rLoss: 0.2879  oError: 0.5106  conv: 0.4605  avgPre: 0.5780
Batch: [0003/0300]  training loss: 1.2328  meta_loss: 1.6842  hLoss: 0.2120  rLoss: 0.2943  oError: 0.5604  conv: 0.4628  avgPre: 0.5611
Batch: [0004/0300]  training loss: 1.1905  meta_loss: 1.4682  hLoss: 0.2174  rLoss: 0.2739  oError: 0.5669  conv: 0.4437  avgPre: 0.5699
Batch: [0005/0300]  training loss: 1.1748  meta_loss: 1.5445  hLoss: 0.2188  rLoss: 0.2754  oError: 0.5443  conv: 0.4359  avgPre: 0.5707
Batch: [0006/0300]  training loss: 1.1273  meta_loss: 1.4998  hLoss: 0.2183  rLoss: 0.2758  oError: 0.5026  conv: 0.4411  avgPre: 0.5872
Batch: [0007/0300]  training loss: 1.1237  meta_loss: 1.5394  hLoss: 0.218

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jan 12 08:32:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

next from mirflickr cv=1 p_true=3

emotions

Best hLoss: 0.2927  Best rLoss: 0.2216  Best oError: 0.3445  Best conv: 0.3697  Best avgPre: 0.7530 p_noise = 9 meta

Best hLoss: 0.2927  Best rLoss: 0.2724  Best oError: 0.3697  Best conv: 0.4062  Best avgPre: 0.7173 p_noise = 9 baseline_clean

Best hLoss: 0.2871  Best rLoss: 0.2653  Best oError: 0.3613  Best conv: 0.4104  Best avgPre: 0.7217 p_noise = 3 baseline_clean

Best hLoss: 0.3235  Best rLoss: 0.2911  Best oError: 0.4202  Best conv: 0.4370  Best avgPre: 0.6867 p_noise = 3 meta

 music_emotion

 Best hLoss: 0.2060  Best rLoss: 0.2424  Best oError: 0.4594  Best conv: 0.4083  Best avgPre: 0.6124 

 Best hLoss: 0.1977  Best rLoss: 0.2524  Best oError: 0.4682  Best conv: 0.4193  Best avgPre: 0.6173 batch = 50 lr = 0.1

 Best hLoss: 0.1958  Best rLoss: 0.2513  Best oError: 0.4660  Best conv: 0.4164  Best avgPre: 0.6181 batch = meta_size lr = 0.01

 Best hLoss: 0.1855  Best rLoss: 0.1984  Best oError: 0.3855  Best conv: 0.3608  Best avgPre: 0.6708 ground_truth

 Best hLoss: 0.1930  Best rLoss: 0.2104  Best oError: 0.4148  Best conv: 0.3705  Best avgPre: 0.6594 ground_truth with relu layer

 Best hLoss: 0.1940  Best rLoss: 0.2204  Best oError: 0.4331  Best conv: 0.3874  Best avgPre: 0.6442 p_true = 9 baseline_clean

 Best hLoss: 0.2044  Best rLoss: 0.2387  Best oError: 0.4674  Best conv: 0.4021  Best avgPre: 0.6132 p_true = 9 baseline_clean creds = plabels

 Best hLoss: 0.2158  Best rLoss: 0.2273  Best oError: 0.4206  Best conv: 0.3900  Best avgPre: 0.6430 p_true = 9 meta


In [ ]:
dir = "/content/drive/MyDrive/Colab Notebooks"

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/result

/content/drive/MyDrive/Colab Notebooks/result


In [ ]:
#methods = ['meta', 'meat_clean', 'baseline', 'baseline_clean', 'ground_truth']
methods = ['meta-ver2']
datasetss = ['music_emotion', 'music_style','mirflickr','YeastBP','emotions','enron','CAL500','scene','genbase']
noise_list = [50,100,150]
#noise_list = [50]
true_list = [1,3,5,7,9]
for method in methods:
  %mkdir {method}
  %cd {method}
  for data in datasetss:
    if data in ['music_emotion', 'music_style','mirflickr','YeastBP']:
      %mkdir {data}
      %cd {data}
      for n in [0]:
        %mkdir {str(n)}
        %cd {str(n)}
        %mkdir true
        %cd true
        for t in true_list:
          %mkdir {str(t)}
        %cd ..
        %cd ..
    else:
      %mkdir {data}
      %cd {data}
      for n in noise_list:
        %mkdir {str(n)}
        %cd {str(n)}
        %mkdir true
        %cd true
        for t in true_list:
          %mkdir {str(t)}
        %cd ..
        %cd ..
    %cd ..
  %cd ..

/content/drive/MyDrive/Colab Notebooks/result/meta-ver2
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/music_emotion
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/music_emotion/0
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/music_emotion/0/true
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/music_emotion/0
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/music_emotion
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/music_style
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/music_style/0
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/music_style/0/true
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/music_style/0
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/music_style
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2
/content/drive/MyDrive/Colab Notebooks/result/meta-ver2/mirflickr
/content/drive/MyDrive/Colab Notebooks/result/meta-ver

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.metrics import hamming_loss, coverage_error, label_ranking_average_precision_score, label_ranking_loss
import numpy as np

In [ ]:
d_list=["emotions","enron","CAL500","scene","genbase","mirflickr","music_style","music_emotion"]
for i in range(len(d_list)):
  dataname=d_list[i]
  ath = f"/content/drive/MyDrive/Colab Notebooks/new_data2/" + dataname + "/"
  labels = np.loadtxt(ath+"target.csv", delimiter=',',dtype =float)
  if dataname in ["mirflickr","music_style","music_emotion"]:
    plabels = np.loadtxt(ath+"cand/0.csv", delimiter=',',dtype =float)
  else:
    plabels = np.loadtxt(ath+"cand/50.csv", delimiter=',',dtype =float)
  a,b=np.shape(labels)
  z=np.zeros((a,b))
  o=np.ones((a,b))
  zl=coverage_error(labels,z)
  ol=coverage_error(labels,o)
  print(dataname)
  print("zl="+str(zl))
  print("ol="+str(ol))

In [ ]:
a = np.loadtxt("/content/drive/MyDrive/Colab Notebooks/result/predict_creds/PML-MD/emotions/50/true/1/creds_0.csv",delimiter = ",")
ba = np.loadtxt("/content/drive/MyDrive/Colab Notebooks/result/predict_creds/PML-MD/emotions/50/true/1/target_0.csv",delimiter = ",")
print(np.shape(a))
print(np.shape(ba))

(470, 6)
(470, 6)
